# IMPORTING NECESSARY LIBRARIES

In [125]:
import os
import re
import nltk
import glob
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [126]:
sw = set(stopwords.words('english'))

# INVERTED INDEX CONSTRUCTION

In [127]:
# inverted index dictionary creation
index = {}
# document id to document name conversion
DOCINFO = {} #contains the information about the document
power_set = [] #this is going to be the power set which is the universe U
one,two = 1,2 #these are the variables which we use for updating the number of comparisons by 1 and 2 respectively
wind = "\\" #for windows, we use this 
ubuntu = "/" #for ubuntu  we use this
active = wind #active means we are using which either windows or ubuntu currently

# OR QUERY

In [128]:
def XORY(x, y): #this returns the number of comparisons and the number of retrieved documents when we perform or operation
    x, y = sorted(x), sorted(y) #we sort both the x and y before doing computations on them, this way we can do it fast
    _OR = [] #this is going to be the array where we will store the words
    comp = 0 #number of cmparisons we will store here.
    i, j = 0, 0 #initiase the iterators i and j to 0 i iterates x and j iterates y
    while i<len(x) and j<len(y):
        if x[i]==y[j]: #first comparison , if this is false, we have to do one more comparison
            _OR.append(y[j]) #if you have found the match, in that case, just simply, append the word in the or array
            j += 1 #i will move one step ahead
            i += 1 #j will move one step ahead
            comp +=one #the number of comparisons we did in this case is simply 1 as we compared x[i] with y[j]
        else: #if there is not a match, in that case, if block is not executed and we have wasted one comparison there.
            comp += two #hence, we increase the comparison count by two.
            if x[i]<y[j]: #so, if x[i] is less than y[j] this means, we have found x[i] so we will append x[i]
                _OR.append(x[i])
                i += 1
            else:
                _OR.append(y[j]) #likewise we will process y[j] and and append y[j] in the array
                j += 1
    while i<len(x): #for all theh remaining array lengths, we will append the array like this
        _OR.append(x[i])
        i += 1 #increment i to move forward
    while j<len(y):
        _OR.append(y[j])
        j += 1
    return _OR, comp

# AND QUERY

In [129]:

def XANDY(x, y):
    x, y = sorted(x), sorted(y)
    _AND = []
    comp = 0
    i, j = 0, 0
    while i<len(x) and j<len(y):
        if x[i] == y[j]:
            _AND.append(y[j])
            j += 1
            i += 1
            comp += one
        else:
            comp += two
            if x[i]<y[j]:
                i+=1
            else:
                j+=1
    return _AND, comp #returning the and array and the number of comparisons we made


# SET DIFFERENCE METHOD IN ORDER TO FIND OR NOT & AND NOT

In [130]:
def XMINUSY(x, y):
    x = sorted(x)
    i, j = 0, 0
    _AminusB = []
    comp = 0
    while i < len(x) and j < len(y):
        if x[i] == y[j]:
            comp += one
            i += 1
            j += 1
        else:
            comp += two
            if x[i] > y[j]:
                _AminusB.append(y[j])
                j += 1
            else:
                i += 1
    while j < len(y):
        _AminusB.append(y[j])
        j += 1
    return _AminusB, comp


# OR NOT QUERY USING SET DIFFERENCE

In [131]:
# x or not y = U-(y-x)
def XORNOTY(x, y):
    xbar, comp1 = XMINUSY(x,y)
    xbarbar, comp2 = XMINUSY(xbar, power_set)
    return xbarbar, comp1+comp2

# AND NOT QUERY USING SET DIFFERENCE

In [132]:
#X^Y'
def XANDNOTY(x, y):
    ans, comp = XMINUSY(y, x)
    return ans, comp

# METHOD TO GET DOCUMENT LIST CORRESPONDING TO A WORD

In [133]:
def get_doc_list_for_word(word):
    if word in index:
        return index[word]
    else:
        var = []
        return var
def get_name_for_doc_id(doc_id):
    if doc_id in DOCINFO:
        return DOCINFO[doc_id]
    else:
        return ""

# EVALUATING THE CLUSTERS 

In [134]:
def evalc(dls, operation=or_query):
    lcpy = dls[:]
    sum_list = {}
    comparisons = 0
    while len(lcpy) != 1:
        sum_list = {}
        for i in range(len(lcpy) - 1):
            sum_list[i] = len(lcpy[i]) + len(lcpy[i + 1])

        min_i = min(sum_list, key=sum_list.get)

        lcpy[min_i], opr = operation(lcpy[min_i], lcpy[min_i + 1])
        comparisons += opr
        del lcpy[min_i + 1]

    fl = lcpy[:][0]
    return fl, comparisons

#  CREATING CLUSTERS

In [135]:
def create_clusters(qdl, ops):
    # && >>> ||
    opfun = {'OR': or_query, 'AND': and_query}
    opers = ops[:]
    dl = qdl[:]
    comparisons = 0

    if 'AND' in opers:
        # Finding AND Clusters
        clust = []
        is_continued = False
        i = 0
        while i < len(opers):
            if opers[i] == 'AND':
                if not is_continued:
                    is_continued = True
                clust.append(dl[i])
                del dl[i]
                del opers[i]
            else:
                if is_continued:
                    clust.append(dl[i])
                    l, comp = evalc(clust, operation=opfun['AND'])
                    clust = []
                    dl[i] = l
                    comparisons += comp
                is_continued = False
                i += 1
        if is_continued:
            clust.append(dl[-1])
            l, comp = evalc(clust, operation=opfun['AND'])
            clust = []
            dl[i] = l
            comparisons += comp

    l, comp = evalc(dl, operation=opfun['OR'])

    return l, comparisons + comp


# PERFORM QUERY

In [136]:
def perform_query(query_words, ops):
    comparisons = 0
    if len(query_words) == 1:
        if len(ops) > 0:
            print('Operations ignored.')
        ids = get_doc_list_for_word(query_words[0])
        return ids, 0, [get_name_for_doc_id(d_id) for d_id in ids]

    if not (len(query_words) == len(ops) + 1):
        raise ValueError('Size mismatch, expected len(qs) = {len_sent} and len(ops) = {len_opr}.'
                         .format(len_sent=len(query_words), len_opr=len(ops)))

    qdl = [0] * len(query_words)
    for i in range(len(query_words)):
        qdl[i] = get_doc_list_for_word(query_words[i])

    dl = qdl[0]
    comp = 0
    for i in range(len(ops)):
        if ops[i] == 'OR':
            dl, comp = XORY(dl, qdl[i + 1])
        elif ops[i] == 'AND':
            dl, comp = XANDY(dl, qdl[i + 1])
        elif ops[i] == 'OR NOT':
            dl, comp = XORNOTY(dl, qdl[i + 1])
        elif ops[i] == 'AND NOT':
            dl, comp = XANDNOTY(dl, qdl[i + 1])

        comparisons += comp

    print(comparisons)

    return dl, comparisons, [get_name_for_doc_id(x) for x in dl]

# UPDATING THE INDEX BASED ON FOLDER,  FILE AND FILE ID

In [137]:

def update_index(folder_name, filename, file_id):
    alpha = ""
    try:
        alpha = open(folder_name + filename, 'r').read()
    # except IOError:
    #     print("This file has some problem in reading ->", filename)
    except UnicodeDecodeError:
        alpha = str(open(folder_name + filename, 'rb').read())

    ppalpha = pp(alpha)  # list form.

    for t in ppalpha:
        if t in index:
            if index[t][-1] < file_id:
                index[t].append(file_id)
        else:
            index[t] = [file_id]

#  PREPARING THE DATASET

In [138]:
def prepare_dataset():
    cwd = os.getcwd()
    filename_with_path = glob.glob(cwd + active + "hhmf" + active + "*")
    F = [i.split(active)[-1] for i in filename_with_path]
    # Indexing each filename.
    i = 0
    # Pre-Processing and updating 'index' dictionary
    for filename in F:
        update_index(cwd + active + "hhmf"  + active , filename, i)
        DOCINFO[i] = filename
        i += 1
    global power_set
    power_set = sorted(list(DOCINFO.keys()))	# [0, 1, 2, ...., 467]

# PREPROCESSING

In [139]:
def DEL_STOPWORDS(qs):
    return [w for w in qs if w not in sw]

def pp(alpha):
    alpha = alpha.lower()
    alpha = re.sub(r'[+*/\\\-?.>,<\"\';:!@#$%^&()_`~]', ' ', alpha)
    alpha = word_tokenize(alpha)
    alpha = [word for word in alpha if word.isalpha()]
    alpha = DEL_STOPWORDS(alpha)
    return alpha


#  MAIN FUNCTION

In [140]:
def main():
    T = int(input('Enter no of queries : ').strip())
    if T == 0:
        return
    print('========== Preparing Dataset ==========', end=' ')
    prepare_dataset()  # index is updated.
    print('[OK]')
    print("Vocabulary :", len(index.keys()), '#Documents :', len(power_set))
    for i in range(T):
        qs = pp(input('\nYou are requested to enter your query : '))
        if len(qs) > 0:
            n = len(qs) - 1
            ops = input('Enter the operation sequence (Only ' + str(n) + ' operations is needed) : ').split(",")
            ops = [x.strip().upper() for x in ops]
            print('Running query : ', end=' ')
            for j in range(len(ops)):
                print(qs[j], end=' ')
                print(ops[j], end=' ')
            print(qs[-1])
            doc_id, minops, doc_names = perform_query(qs, ops)
            print("Document ID : ", doc_id)
            print("Document Names (10 max) : ", doc_names[:10])
            print("Number of documents matched: ", len(doc_id))
            print("Minimum Operation : ", minops)
        else:
            print('Nothing to process')
if "__main__":
    main()


Enter no of queries : 1
========== Preparing Dataset ========== [OK]
Vocabulary : 65185 #Documents : 1133

You are requested to enter your query : the lion stood thoughtfully for a moment
Enter the operation sequence (Only 3 operations is needed) : [OR,OR,OR]
Running query :  lion [OR stood OR thoughtfully OR] moment
96
Document ID :  [116, 135, 138, 218, 241, 248, 291, 295, 318, 322, 378, 418, 533, 595, 596, 597, 602, 635, 688, 689, 753, 791, 842, 977, 980, 1027, 1039]
Document Names (10 max) :  ['bitnet.txt', 'boneles2.txt', 'booze1.fun', 'christop.int', 'collected_quotes.txt', 'computer.txt', 'deep.txt', 'devils.jok', 'dromes.txt', 'dthought.txt']
Number of documents matched:  27
Minimum Operation :  96
